# Training BYOC on SageMaker
>__Note:__ This assumes a container has already been built and deployed to ECR.

## Prepare the Data

In [1]:
# Load the required libraries
import warnings
import zipfile
import boto3
import os
import urllib.request
import sagemaker
import tempfile
import cv2
import numpy as np
import pandas as pd
import matplotlib.image as mpimg
from sagemaker.estimator import Estimator
from sklearn.model_selection import train_test_split

In [2]:
region = boto3.Session().region_name
sagemaker_session = sagemaker.Session()
sagemaker_client = boto3.client('sagemaker')
bucket = sagemaker.Session().default_bucket()
role = sagemaker.get_execution_role()

# MXNet GPU

## Set Hyperparameters

In [3]:
channels = {'train': 's3://robostig-assets-us-west-2'}
training_instance_type = 'ml.p3.2xlarge'
#training_instance_type = 'ml.m5.24xlarge'
hyperparameters = dict(batch_size=32, learning_rate=.0001, epochs=12, gpu_count=1)
output_location = "s3://{}".format(bucket)

In [4]:
image_name = 'mx-gpu'
BYOC_estimator = Estimator(
    image_name,
    role=role,
    output_path=output_location,
    train_instance_count=1,
    train_instance_type=training_instance_type,
    hyperparameters=hyperparameters,
    sagemaker_session=sagemaker_session
)

In [5]:
BYOC_estimator.fit(channels)

INFO:sagemaker:Creating training-job with name: pystig-2018-07-09-17-28-59-745


.................................
Using MXNet backend
Starting model training ...
Hyper parameters: {'epochs': '12', 'batch_size': '32', 'learning_rate': '0.0001', 'gpu_count': '1'}
Input parameters: {'train': {'RecordWrapperType': 'None', 'TrainingInputMode': 'File', 'S3DistributionType': 'FullyReplicated'}}

comma.ai Model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (0, 3, 66, 200)           0         
_________________________________________________________________
conv2d_1 (Conv2D)            (0, 16, 17, 50)           3088      
_________________________________________________________________
activation_1 (Activation)    (0, 16, 17, 50)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (0, 32, 9, 25)            12832     
_________________________________________________________________
activation_2

226/226 [==============================] - 49s 217ms/step - loss: 0.0420 - val_loss: 0.0174
Epoch 2/12

 63/226 [=======>......................] - ETA: 24s - loss: 0.0346


176/226 [======================>.......] - ETA: 8s - loss: 0.0346


226/226 [==============================] - 40s 178ms/step - loss: 0.0339 - val_loss: 0.0212
Epoch 3/12

119/226 [==============>...............] - ETA: 17s - loss: 0.0333


...

176/226 [======================>.......] - ETA: 8s - loss: 0.0246


226/226 [==============================] - 40s 178ms/step - loss: 0.0246 - val_loss: 0.0182
Epoch 11/12

 90/226 [==========>...................] - ETA: 21s - loss: 0.0233


203/226 [=========================>....] - ETA: 3s - loss: 0.0234


226/226 [==============================] - 40s 178ms/step - loss: 0.0238 - val_loss: 0.0143
Epoch 12/12

145/226 [==================>...........] - ETA: 13s - loss: 0.0231


226/226 [==============================] - 40s 178ms/step - loss: 0.0225 - val_loss: 0.0208
Saving model ...
===== Job Complete =====
Billable seconds: 744


# TensorFlow GPU


In [6]:
hyperparameters = dict(batch_size=32, learning_rate=.0001, epochs=12)
image_name = '*tf-gpu'
BYOC_estimator = Estimator(
    image_name,
    role=role,
    output_path=output_location,
    train_instance_count=1,
    train_instance_type='ml.p3.2xlarge',
    hyperparameters=hyperparameters,
    sagemaker_session=sagemaker_session
)

In [7]:
BYOC_estimator.fit(channels)

INFO:sagemaker:Creating training-job with name: pystig-2018-07-09-17-43-26-229


..................................
Using TensorFlow backend.
creating SageMaker trainer environment:
TrainerEnvironment(input_dir='/opt/ml/input', input_config_dir='/opt/ml/input/config', model_dir='/opt/ml/model', output_dir='/opt/ml/output', hyperparameters={'epochs': '12', 'learning_rate': '0.0001', 'batch_size': '32'}, resource_config={'hosts': ['algo-1'], 'network_interface_name': 'ethwe', 'current_host': 'algo-1'}, input_data_config={'train': {'TrainingInputMode': 'File', 'RecordWrapperType': 'None', 'S3DistributionType': 'FullyReplicated'}}, output_data_dir='/opt/ml/output/data', hosts=['algo-1'], channel_dirs={'train': '/opt/ml/input/data/train'}, current_host='algo-1', available_gpus=1, available_cpus=8)
Starting model training ...

comma.ai Model Summary

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 66, 200, 3)        0         
__________________________

174/226 [======================>.......] - ETA: 11s - loss: 0.0535


226/226 [==============================] - 51s 227ms/step - loss: 0.0517 - val_loss: 0.0177
Epoch 2/12

118/226 [==============>...............] - ETA: 17s - loss: 0.0425


226/226 [==============================] - 41s 180ms/step - loss: 0.0411 - val_loss: 0.0180
Epoch 3/12

 36/226 [===>..........................] - ETA: 24s - loss: 0.0338


...

226/226 [==============================] - 41s 179ms/step - loss: 0.0321 - val_loss: 0.0161
Epoch 10/12

146/226 [==================>...........] - ETA: 13s - loss: 0.0315


226/226 [==============================] - 41s 179ms/step - loss: 0.0305 - val_loss: 0.0183
Epoch 11/12

 93/226 [===========>..................] - ETA: 21s - loss: 0.0293


206/226 [==========================>...] - ETA: 3s - loss: 0.0293


226/226 [==============================] - 40s 179ms/step - loss: 0.0291 - val_loss: 0.0188
Epoch 12/12

119/226 [==============>...............] - ETA: 17s - loss: 0.0297


226/226 [==============================] - 40s 179ms/step - loss: 0.0298 - val_loss: 0.0196
Saving the trained model ...
===== Job Complete =====
Billable seconds: 758


## Training Job Description

In [ ]:
response = sagemaker_client.describe_training_job(
    TrainingJobName='pystig-2018-07-08-03-54-44-356'
)
response

---
## Deploy model - Standard
__Use `estimator.deploy()` based on GPU Container training__
>__Note:__ This is not cost effective.

```
predictor = BYOC_estimator.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge')
```

---
## Deploy Model - CPU Container
__Use separate CPU container and the `sagemaker.Session()` API to specify a different serving container__

### Step 1: Create a new model from the training job, specifying a different container for training, in this case a CPU-based container.

In [ ]:
job_name = 'pystig-2018-07-08-03-54-44-356'
BYOC_model = sagemaker_session.create_model_from_job(
    name = job_name.split('-')[0]+'-model',
    training_job_name=job_name,
    role=role,
    primary_container_image='500842391574.dkr.ecr.us-west-2.amazonaws.com/pystig:cpu',
    model_data_url='s3:/{}/{}/output/model.tar.gz'.format(bucket, job_name)
)

### Step 2: Create a SageMaker Endpoint Configuration

In [ ]:
BYOC_endpoint_config_name = sagemaker_session.create_endpoint_config(
    name=job_name.split('-')[0]+'-endpoint-config',
    model_name=BYOC_model,
    initial_instance_count=1,
    instance_type='ml.c4.xlarge'
)

### Step 3: Deploy the SageMaker Endpoint

In [ ]:
create_endpoint_response = sagemaker_session.create_endpoint(
    endpoint_name=job_name.split('-')[0]+'-endpoint',
    config_name=str(BYOC_endpoint_config_name)
)

In [ ]:
sagemaker_client.describe_endpoint(EndpointName=create_endpoint_response)

---
## Test Endpoint (Simuilate pyStig)
### Get Sample Data for predictions

In [ ]:
# Helper functions
def download(url):
    """
    Helper function to download individual file from given url.
    
    Arguments:
    url -- full URL of the file to download
    
    Returns:
    filename -- downloaded file name
    """
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)
    return filename

# To download and extract Sample Data
file = download('https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip')

# Extract the file
with zipfile.ZipFile(file) as zf:
    zf.extractall()
    
# Image Transofmrations
def crop(image):
    """
    Crop the image (removing the sky at the top and the car front at the bottom).
    
    Returns:
    Cropped image.
    """
    return image[60:-25, :, :]

def resize(image):
    """
    Resize the image to the input shape used by the network model.
    
    Returns:
    Resized image.
    """
    return cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), cv2.INTER_AREA)

def rgb2yuv(image):
    """
    Convert the image from RGB to YUV.
    
    Returns:
    YUV image.
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2YUV)

def load(data_dir, image_file):
    """
    Load RGB images from a file
    """
    return mpimg.imread(os.path.join(data_dir, image_file.strip()))

def transform(image):
    """
    Combine all preprocess functions into one
    """
    image = crop(image)
    image = resize(image)
    image = rgb2yuv(image)
    return image

### Random Sample Image

In [ ]:
# Origional 'left' image
IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 66, 200, 3
INPUT_SHAPE = (IMAGE_CHANNELS, IMAGE_HEIGHT, IMAGE_WIDTH)
data_df = pd.read_csv('./data/driving_log.csv')
X = data_df[['center', 'left', 'right']].values
y = data_df['steering'].values
random_image = X[100][0]
img = load('data', random_image)

In [ ]:
img.shape

In [ ]:
img = img.reshape(img.shape[2], img.shape[0], img.shape[1])

In [ ]:
img.shape

In [ ]:
normalize = lambda x: x/127.55 -1
print(normalize(img).shape)

In [ ]:
# Simulate pyStig call by first pre-preocessing image and converting to 4D array
endpoint_name = sagemaker_client.describe_endpoint(EndpointName=create_endpoint_response)['EndpointName']
payload = np.array([transform(img)])

In [ ]:
runtime_client = boto3.client('sagemaker-runtime')
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload.tolist())
)
prediction = float(json.loads(response['Body'].read().decode('utf-8'))[0])

In [ ]:
prediction

In [ ]:
y[100]